In [1]:
import numpy as np
import allel
import seaborn as sns
import pandas as pd
import sys
sys.path.append('../../../')
from mxbgenomes.utils import load_populations_info

In [2]:
# Removing correlated features (LD pruning)
def ld_prune(gn, size, step, threshold=.1, n_iter=1):
    for i in range(n_iter):
        loc_unlinked = allel.locate_unlinked(gn, size=size, step=step, threshold=threshold)
        n = np.count_nonzero(loc_unlinked)
        n_remove = gn.shape[0] - n
        print('iteration', i+1, 'retaining', n, 'removing', n_remove, 'variants')
        gn = gn.compress(loc_unlinked, axis=0)
    return gn


def pca_projection(g, sample_names):
    """
    Args:
        g: Genotype array
        sample_naames: list of samples names
    """
    ac = g.count_alleles()
    # filter multiallelic snps and singletons
    flt = (ac.max_allele() == 1) & (ac[:, :2].min(axis=1) > 1)
    gf = g.compress(flt, axis=0)
    
    # transform the genotype data into a 2-dimensional matrix where 
    # each cell has the number of non-reference alleles
    # per call. This is what we’ll use as the input to PCA.
    gn = gf.to_n_alt()
    gnu = ld_prune(gn, size=500, step=200, threshold=.1, n_iter=1)
    
    # pca decomposition
    coords1, model1 = allel.pca(gnu, n_components=10, scaler='patterson')
    colnames = ['PC_' + str(x) for x in range(1, 11)]
    coord_pca = pd.DataFrame(coords1, columns=colnames)
    coord_pca['Samplename'] = sample_names
    
    print(model1.explained_variance_ratio_[0]*100)
    print(model1.explained_variance_ratio_[1]*100)
    print(model1.explained_variance_ratio_[2]*100)
    return coord_pca


In [24]:
## Real data
# Samples to use for PCA analysis
popinfo = load_populations_info('../../../')
popinfo = popinfo[popinfo.Subpopulation.isin(['MXL', 'CHB', 'YRI', 'IBS'])]
samples = popinfo.Samplename.tolist()

# read the vcf faile for chr1
vcf = allel.read_vcf('../../../results/data/210713-HardyW-filters/1TGP_and_50MXB-chr22-snps-vep-mask-HW-GRCh38.vcf.gz')


# subset the samples in the VCF
samples_vcf = [x for x in vcf['samples'] if x in samples]
samples_vcf_indicator = [x in samples for x in vcf['samples']]
samples
vcf_real = vcf['calldata/GT'][:, samples_vcf_indicator, :]


In [28]:
g = allel.GenotypeArray(vcf_real)
pca = pca_projection(g, samples)

iteration 1 retaining 33594 removing 163153 variants
2.3930231109261513
1.440543495118618
0.6787079386413097


In [32]:
pca.merge(popinfo).to_csv('./results/pca-real.csv', index=False)

In [33]:
## Browning
vcf = allel.read_vcf('data/BrowningEtAl2011.vcf')
g = allel.GenotypeArray(vcf['calldata/GT'])
pca = pca_projection(g, vcf['samples'])
pca.to_csv('./results/pca-BrowningEtAl2011.csv', index=False)

iteration 1 retaining 26487 removing 131386 variants
1.6682503744959831
1.0838383808732033
0.5033210851252079


In [4]:
## Medina 2022
def is_from_pop(x):
    return x.split('_')[0] in ['YRI', 'IBS', 'CHB', 'MXL']

vcf = allel.read_vcf('../220202-SimulatePCA/data/simulated-genomes-chr22.vcf')
samples_vcf = [x for x in vcf['samples'] if is_from_pop(x)]
samples_vcf_indicator = [x in samples_vcf for x in vcf['samples']]

g = allel.GenotypeArray(vcf['calldata/GT'][:, samples_vcf_indicator, :])

In [ ]:
pca = pca_projection(g, samples_vcf)
pca.to_csv('./results/pca-Medina2022.csv', index=False)